In [127]:
#!/usr/bin/python3

# Webserver dependencies
from http.server import BaseHTTPRequestHandler, HTTPServer
import argparse
import time
import google.cloud.storage as storage
import google.cloud.pubsub as pubsub

# Sql dependencies
import os
from google.cloud.sql.connector import Connector, IPTypes
import pymysql
import socket, struct
import sqlalchemy

import numpy as np
import pandas as pd

from sklearn import linear_model
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [128]:
PROJECT_ID = "feisty-gasket-398719"
TOPIC_ID = "my-topic"
SUBSCRIPTION_NAME = "my-topic-sub"
INSTANCE_CONNECTION_NAME = "feisty-gasket-398719:us-east1:instance-tigeryi"
DB_USER = "root"
DB_PASS = ""
DB_NAME = "dbhw5"
DB_PRIVATE_IP = False
BANNED_COUNTRIES = ["North Korea", "Iran", "Cuba", "Myanmar", "Iraq", "Libya", "Sudan", "Zimbabwe", "Syria"]
HTTP_METHODS = ['GET', 'HEAD', 'POST', 'PUT', 'DELETE', 'CONNECT', 'OPTIONS', 'TRACE', 'PATCH']

In [129]:
def connect_with_connector() -> sqlalchemy.engine.base.Engine:
        """
        Initializes a connection pool for a Cloud SQL instance of MySQL.
        
        Uses the Cloud SQL Python Connector package.
        """
        # Note: Saving credentials in environment variables is convenient, but not
        # secure - consider a more secure solution such as
        # Cloud Secret Manager (https://cloud.google.com/secret-manager) to help
        # keep secrets safe.

        instance_connection_name = INSTANCE_CONNECTION_NAME # os.environ["INSTANCE_CONNECTION_NAME"]  # e.g. 'project:region:instance'
        db_user = DB_USER # os.environ["DB_USER"]  # e.g. 'my-db-user'
        db_pass = DB_PASS # os.environ["DB_PASS"]  # e.g. 'my-db-password'
        db_name = DB_NAME # os.environ["DB_NAME"]  # e.g. 'my-database'

        ip_type = IPTypes.PRIVATE if DB_PRIVATE_IP else IPTypes.PUBLIC # if os.environ.get("DB_PRIVATE_IP")

        connector = Connector(ip_type)

        def getconn() -> pymysql.connections.Connection:
            conn: pymysql.connections.Connection = connector.connect(
                instance_connection_name,
                "pymysql",
                user=db_user,
                password=db_pass,
                db=db_name,
            )
            return conn
        
        pool = sqlalchemy.create_engine(
            "mysql+pymysql://",
            creator=getconn,
            # ...
        )
        return pool

In [130]:
pool = connect_with_connector()

In [131]:
query = '''select ip2, country2, ip, country from table2;'''


In [142]:
df = pd.read_sql_query(query, pool)

In [143]:
print(df.head())

          ip2  country2              ip      country
0  2737397699       160   163.41.95.195      Somalia
1  2509268020        89  149.144.100.52       Kosovo
2  1314065103       113    78.83.10.207       Monaco
3  2436094690         9  145.51.218.226      Austria
4   294390865       162    17.140.12.81  South Korea


In [144]:
len(df)

72971

In [145]:
X = df[['ip2']]
X

,ip2
0,2737397699
1,2509268020
2,1314065103
3,2436094690
4,294390865
...,...
72966,2357763574
72967,1558140856
72968,2638169342
72969,3722494499


In [146]:
y = df['country2']
y

0        160
1         89
2        113
3          9
4        162
        ... 
72966    181
72967     97
72968     36
72969     28
72970    181
Name: country2, Length: 72971, dtype: int64

In [147]:
clf = DecisionTreeClassifier() 

In [148]:
clf.fit(X,y)

DecisionTreeClassifier()

In [149]:
clf.score(X,y)

1.0

In [150]:
y_pred = clf.predict(X)
y_pred

array([160,  89, 113, ...,  36,  28, 181])

In [151]:
accuracy_score(y, y_pred)

1.0